In [2]:
import pandas as pd
import time
import re
import gc
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [3]:
df_original = pd.read_csv("RocketPunch.csv")

In [4]:
df_original.head()

,name,url
0,렌팃,https://www.rocketpunch.com/companies/rentit
1,오토블링,https://www.rocketpunch.com/companies/theautob...
2,21그램,https://www.rocketpunch.com/companies/21gram-1
3,Kpopmap,https://www.rocketpunch.com/companies/kpopmap
4,잇다,https://www.rocketpunch.com/companies/itda-3


In [5]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29139 entries, 0 to 29138
Data columns (total 2 columns):
name    29134 non-null object
url     29139 non-null object
dtypes: object(2)
memory usage: 227.7+ KB


In [6]:
df = pd.DataFrame(df_original.groupby("name")['url'].apply(list))
df.reset_index(inplace = True)
df['url'] = df.url.apply(lambda x: x[0])

df.head(3)

,name,url
0,#Jinair,https://www.rocketpunch.com/companies/jinair-1
1,#Sol,https://www.rocketpunch.com/companies/solkorealtd
2,'윌슨'프로젝트,https://www.rocketpunch.com/companies/yebicang...


## 웹스크래핑 (정보추가)

In [7]:
df['date_found'] = 0     #설립입
df['number_people'] = 0  #구성원
df['number_invest'] = 0  #투자유치
df['series'] = 0         #투자레벨
df['location'] = 0       #사무실주소

In [8]:
df.head(2)

,name,url,date_found,number_people,number_invest,series,location
0,#Jinair,https://www.rocketpunch.com/companies/jinair-1,0,0,0,0,0
1,#Sol,https://www.rocketpunch.com/companies/solkorealtd,0,0,0,0,0


In [9]:
df_info = df[['date_found', 'number_people', 'number_invest', 'series', 'location']]
df.drop(['date_found', 'number_people', 'number_invest', 'series', 'location'], axis = 1, inplace = True)

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

In [12]:
def info_scrape(url,n):
    
    global driver
    
    d_found = 0
    n_people = 0
    n_invest = 0
    location = 0
    lv_invest = 0
    
    driver.get(url)
    time.sleep(0.5)
    html = driver.page_source
    bs_xml = bs(html, 'html.parser')
    cp_info = bs_xml.find('div', {'class':'ui divided company info items'})
    
    try:
        str_html = cp_info.get_text()
    except :
        print("빈 페이지")
        return [d_found, n_people, n_invest, lv_invest, location]
    
    if re.search('[0-9]+-[0-9]+-[0-9]+', str_html) != None:
        d_found = re.findall('[0-9]+-[0-9]+-[0-9]+', str_html)[0]
    if re.search('[0-9]+-[0-9]+명', str_html) != None:
        n_people = re.findall('[0-9]+-[0-9]+명', str_html)[0]
    if re.search('대한민국.+', cp_info.get_text()) != None:
        location = re.findall('대한민국.+', str_html)[0]
    if re.search('.+만 원', cp_info.get_text()) != None:
        n_invest = re.findall('.+만 원', str_html)[0].strip()
    if re.search('Series [a-zA-Z]', bs_xml.get_text()) != None:
        lv_invest = re.findall('Series [a-zA-Z]', bs_xml.get_text())[0]
    return [d_found, n_people, n_invest, lv_invest, location]

In [19]:
driver = webdriver.Chrome('C:/chromedriver.exe', chrome_options=options)
for _ in range(len(df.index)):
    cp_url = df.iloc[_]['url']
    df_info.iloc[_] = info_scrape(cp_url, _)
    print(_,"번 째 기업 완료.")
    if _ % 10 == 0:
        gc.collect()
        print("메모리청소")
driver.close()

빈 페이지


C:\Users\813034\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0 번 째 기업 완료.
메모리청소
1 번 째 기업 완료.
2 번 째 기업 완료.
3 번 째 기업 완료.
4 번 째 기업 완료.
빈 페이지
5 번 째 기업 완료.
6 번 째 기업 완료.
빈 페이지
7 번 째 기업 완료.
빈 페이지
8 번 째 기업 완료.
9 번 째 기업 완료.
10 번 째 기업 완료.
메모리청소
11 번 째 기업 완료.
12 번 째 기업 완료.
13 번 째 기업 완료.
14 번 째 기업 완료.
15 번 째 기업 완료.
빈 페이지
16 번 째 기업 완료.
17 번 째 기업 완료.
18 번 째 기업 완료.
빈 페이지
19 번 째 기업 완료.
빈 페이지
20 번 째 기업 완료.
메모리청소
21 번 째 기업 완료.
빈 페이지
22 번 째 기업 완료.
23 번 째 기업 완료.
빈 페이지
24 번 째 기업 완료.
빈 페이지
25 번 째 기업 완료.
26 번 째 기업 완료.
27 번 째 기업 완료.
빈 페이지
28 번 째 기업 완료.
29 번 째 기업 완료.
빈 페이지
30 번 째 기업 완료.
메모리청소
31 번 째 기업 완료.
빈 페이지
32 번 째 기업 완료.
빈 페이지
33 번 째 기업 완료.
빈 페이지
34 번 째 기업 완료.
빈 페이지
35 번 째 기업 완료.
빈 페이지
36 번 째 기업 완료.
37 번 째 기업 완료.
38 번 째 기업 완료.
빈 페이지
39 번 째 기업 완료.
40 번 째 기업 완료.
메모리청소
41 번 째 기업 완료.
빈 페이지
42 번 째 기업 완료.
빈 페이지
43 번 째 기업 완료.
빈 페이지
44 번 째 기업 완료.
45 번 째 기업 완료.
빈 페이지
46 번 째 기업 완료.
47 번 째 기업 완료.
48 번 째 기업 완료.
49 번 째 기업 완료.
50 번 째 기업 완료.
메모리청소
51 번 째 기업 완료.
빈 페이지
52 번 째 기업 완료.
빈 페이지
53 번 째 기업 완료.
54 번 째 기업 완료.
55 번 째 기업 완료.
빈 페이지
56 번 째 기업 완료.
57 번 째 기업 완료.
58 번 째 기업 완료.
빈 페이

C:\Users\813034\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\813034\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


66 번 째 기업 완료.
67 번 째 기업 완료.
빈 페이지
68 번 째 기업 완료.
69 번 째 기업 완료.
70 번 째 기업 완료.
메모리청소
71 번 째 기업 완료.
빈 페이지
72 번 째 기업 완료.
빈 페이지
73 번 째 기업 완료.
빈 페이지
74 번 째 기업 완료.
75 번 째 기업 완료.
76 번 째 기업 완료.
빈 페이지
77 번 째 기업 완료.
78 번 째 기업 완료.
79 번 째 기업 완료.
빈 페이지
80 번 째 기업 완료.
메모리청소
빈 페이지
81 번 째 기업 완료.
빈 페이지
82 번 째 기업 완료.
83 번 째 기업 완료.
84 번 째 기업 완료.
빈 페이지
85 번 째 기업 완료.
86 번 째 기업 완료.
87 번 째 기업 완료.
빈 페이지
88 번 째 기업 완료.
89 번 째 기업 완료.
빈 페이지
90 번 째 기업 완료.
메모리청소
91 번 째 기업 완료.
92 번 째 기업 완료.
빈 페이지
93 번 째 기업 완료.
94 번 째 기업 완료.
95 번 째 기업 완료.
96 번 째 기업 완료.
97 번 째 기업 완료.
98 번 째 기업 완료.
빈 페이지
99 번 째 기업 완료.
100 번 째 기업 완료.
메모리청소
101 번 째 기업 완료.
102 번 째 기업 완료.
빈 페이지
103 번 째 기업 완료.
104 번 째 기업 완료.
105 번 째 기업 완료.
빈 페이지
106 번 째 기업 완료.
빈 페이지
107 번 째 기업 완료.
108 번 째 기업 완료.
109 번 째 기업 완료.
110 번 째 기업 완료.
메모리청소
111 번 째 기업 완료.
112 번 째 기업 완료.
113 번 째 기업 완료.
114 번 째 기업 완료.
115 번 째 기업 완료.
빈 페이지
116 번 째 기업 완료.
117 번 째 기업 완료.
118 번 째 기업 완료.
빈 페이지
119 번 째 기업 완료.
120 번 째 기업 완료.
메모리청소
빈 페이지
121 번 째 기업 완료.
122 번 째 기업 완료.
123 번 째 기업 완료.
124 번 째 기업 완료.

...

17228 번 째 기업 완료.
빈 페이지
17229 번 째 기업 완료.
17230 번 째 기업 완료.
메모리청소
빈 페이지
17231 번 째 기업 완료.
17232 번 째 기업 완료.
17233 번 째 기업 완료.
빈 페이지
17234 번 째 기업 완료.
빈 페이지
17235 번 째 기업 완료.
17236 번 째 기업 완료.
17237 번 째 기업 완료.
17238 번 째 기업 완료.
빈 페이지
17239 번 째 기업 완료.
17240 번 째 기업 완료.
메모리청소
17241 번 째 기업 완료.
17242 번 째 기업 완료.
17243 번 째 기업 완료.
17244 번 째 기업 완료.
17245 번 째 기업 완료.
17246 번 째 기업 완료.
17247 번 째 기업 완료.
17248 번 째 기업 완료.
17249 번 째 기업 완료.
17250 번 째 기업 완료.
메모리청소
17251 번 째 기업 완료.
17252 번 째 기업 완료.
17253 번 째 기업 완료.
17254 번 째 기업 완료.
17255 번 째 기업 완료.
17256 번 째 기업 완료.
17257 번 째 기업 완료.
17258 번 째 기업 완료.
17259 번 째 기업 완료.
17260 번 째 기업 완료.
메모리청소
17261 번 째 기업 완료.
17262 번 째 기업 완료.
17263 번 째 기업 완료.
17264 번 째 기업 완료.
17265 번 째 기업 완료.
17266 번 째 기업 완료.
17267 번 째 기업 완료.
17268 번 째 기업 완료.
17269 번 째 기업 완료.
17270 번 째 기업 완료.
메모리청소
17271 번 째 기업 완료.
17272 번 째 기업 완료.
17273 번 째 기업 완료.
17274 번 째 기업 완료.
17275 번 째 기업 완료.
17276 번 째 기업 완료.
17277 번 째 기업 완료.
17278 번 째 기업 완료.
17279 번 째 기업 완료.
17280 번 째 기업 완료.
메모리청소
빈 페이지
17281 번 째 기업 완료.
17282 번 째 

17681 번 째 기업 완료.
17682 번 째 기업 완료.
17683 번 째 기업 완료.
17684 번 째 기업 완료.
17685 번 째 기업 완료.
17686 번 째 기업 완료.
17687 번 째 기업 완료.
17688 번 째 기업 완료.
17689 번 째 기업 완료.
빈 페이지
17690 번 째 기업 완료.
메모리청소
17691 번 째 기업 완료.
17692 번 째 기업 완료.
17693 번 째 기업 완료.
빈 페이지
17694 번 째 기업 완료.
17695 번 째 기업 완료.
17696 번 째 기업 완료.
17697 번 째 기업 완료.
17698 번 째 기업 완료.
17699 번 째 기업 완료.
17700 번 째 기업 완료.
메모리청소
17701 번 째 기업 완료.
17702 번 째 기업 완료.
17703 번 째 기업 완료.
17704 번 째 기업 완료.
17705 번 째 기업 완료.
17706 번 째 기업 완료.
17707 번 째 기업 완료.
17708 번 째 기업 완료.
17709 번 째 기업 완료.
17710 번 째 기업 완료.
메모리청소
빈 페이지
17711 번 째 기업 완료.
17712 번 째 기업 완료.
17713 번 째 기업 완료.
17714 번 째 기업 완료.
빈 페이지
17715 번 째 기업 완료.
17716 번 째 기업 완료.
17717 번 째 기업 완료.
빈 페이지
17718 번 째 기업 완료.
17719 번 째 기업 완료.
17720 번 째 기업 완료.
메모리청소
17721 번 째 기업 완료.
17722 번 째 기업 완료.
17723 번 째 기업 완료.
17724 번 째 기업 완료.
17725 번 째 기업 완료.
빈 페이지
17726 번 째 기업 완료.
17727 번 째 기업 완료.
빈 페이지
17728 번 째 기업 완료.
17729 번 째 기업 완료.
17730 번 째 기업 완료.
메모리청소
빈 페이지
17731 번 째 기업 완료.
17732 번 째 기업 완료.
17733 번 째 기업 완료.
빈 페이지
17734 번 째 기업 완료

18127 번 째 기업 완료.
빈 페이지
18128 번 째 기업 완료.
18129 번 째 기업 완료.
18130 번 째 기업 완료.
메모리청소
18131 번 째 기업 완료.
18132 번 째 기업 완료.
18133 번 째 기업 완료.
18134 번 째 기업 완료.
18135 번 째 기업 완료.
18136 번 째 기업 완료.
18137 번 째 기업 완료.
18138 번 째 기업 완료.
18139 번 째 기업 완료.
18140 번 째 기업 완료.
메모리청소
18141 번 째 기업 완료.
18142 번 째 기업 완료.
18143 번 째 기업 완료.
18144 번 째 기업 완료.
18145 번 째 기업 완료.
18146 번 째 기업 완료.
18147 번 째 기업 완료.
18148 번 째 기업 완료.
18149 번 째 기업 완료.
18150 번 째 기업 완료.
메모리청소
18151 번 째 기업 완료.
18152 번 째 기업 완료.
18153 번 째 기업 완료.
18154 번 째 기업 완료.
빈 페이지
18155 번 째 기업 완료.
18156 번 째 기업 완료.
18157 번 째 기업 완료.
18158 번 째 기업 완료.
18159 번 째 기업 완료.
18160 번 째 기업 완료.
메모리청소
18161 번 째 기업 완료.
18162 번 째 기업 완료.
18163 번 째 기업 완료.
18164 번 째 기업 완료.
18165 번 째 기업 완료.
18166 번 째 기업 완료.
18167 번 째 기업 완료.
18168 번 째 기업 완료.
18169 번 째 기업 완료.
18170 번 째 기업 완료.
메모리청소
18171 번 째 기업 완료.
18172 번 째 기업 완료.
18173 번 째 기업 완료.
빈 페이지
18174 번 째 기업 완료.
빈 페이지
18175 번 째 기업 완료.
18176 번 째 기업 완료.
18177 번 째 기업 완료.
18178 번 째 기업 완료.
18179 번 째 기업 완료.
빈 페이지
18180 번 째 기업 완료.
메모리청소
빈 페이지
18181 번 째 

18572 번 째 기업 완료.
18573 번 째 기업 완료.
18574 번 째 기업 완료.
18575 번 째 기업 완료.
18576 번 째 기업 완료.
빈 페이지
18577 번 째 기업 완료.
18578 번 째 기업 완료.
18579 번 째 기업 완료.
빈 페이지
18580 번 째 기업 완료.
메모리청소
18581 번 째 기업 완료.
18582 번 째 기업 완료.
18583 번 째 기업 완료.
빈 페이지
18584 번 째 기업 완료.
18585 번 째 기업 완료.
18586 번 째 기업 완료.
18587 번 째 기업 완료.
18588 번 째 기업 완료.
18589 번 째 기업 완료.
빈 페이지
18590 번 째 기업 완료.
메모리청소
18591 번 째 기업 완료.
18592 번 째 기업 완료.
18593 번 째 기업 완료.
빈 페이지
18594 번 째 기업 완료.
18595 번 째 기업 완료.
빈 페이지
18596 번 째 기업 완료.
18597 번 째 기업 완료.
빈 페이지
18598 번 째 기업 완료.
18599 번 째 기업 완료.
18600 번 째 기업 완료.
메모리청소
18601 번 째 기업 완료.
18602 번 째 기업 완료.
18603 번 째 기업 완료.
빈 페이지
18604 번 째 기업 완료.
18605 번 째 기업 완료.
빈 페이지
18606 번 째 기업 완료.
빈 페이지
18607 번 째 기업 완료.
18608 번 째 기업 완료.
빈 페이지
18609 번 째 기업 완료.
18610 번 째 기업 완료.
메모리청소
18611 번 째 기업 완료.
18612 번 째 기업 완료.
18613 번 째 기업 완료.
18614 번 째 기업 완료.
18615 번 째 기업 완료.
18616 번 째 기업 완료.
18617 번 째 기업 완료.
18618 번 째 기업 완료.
18619 번 째 기업 완료.
18620 번 째 기업 완료.
메모리청소
18621 번 째 기업 완료.
18622 번 째 기업 완료.
18623 번 째 기업 완료.
18624 번 째 기업 완료.
186

19015 번 째 기업 완료.
19016 번 째 기업 완료.
19017 번 째 기업 완료.
19018 번 째 기업 완료.
19019 번 째 기업 완료.
19020 번 째 기업 완료.
메모리청소
19021 번 째 기업 완료.
19022 번 째 기업 완료.
19023 번 째 기업 완료.
19024 번 째 기업 완료.
19025 번 째 기업 완료.
19026 번 째 기업 완료.
19027 번 째 기업 완료.
19028 번 째 기업 완료.
19029 번 째 기업 완료.
19030 번 째 기업 완료.
메모리청소
19031 번 째 기업 완료.
빈 페이지
19032 번 째 기업 완료.
19033 번 째 기업 완료.
19034 번 째 기업 완료.
19035 번 째 기업 완료.
19036 번 째 기업 완료.
빈 페이지
19037 번 째 기업 완료.
19038 번 째 기업 완료.
19039 번 째 기업 완료.
19040 번 째 기업 완료.
메모리청소
19041 번 째 기업 완료.
19042 번 째 기업 완료.
19043 번 째 기업 완료.
19044 번 째 기업 완료.
19045 번 째 기업 완료.
빈 페이지
19046 번 째 기업 완료.
빈 페이지
19047 번 째 기업 완료.
19048 번 째 기업 완료.
19049 번 째 기업 완료.
19050 번 째 기업 완료.
메모리청소
빈 페이지
19051 번 째 기업 완료.
19052 번 째 기업 완료.
19053 번 째 기업 완료.
19054 번 째 기업 완료.
19055 번 째 기업 완료.
19056 번 째 기업 완료.
19057 번 째 기업 완료.
19058 번 째 기업 완료.
19059 번 째 기업 완료.
19060 번 째 기업 완료.
메모리청소
19061 번 째 기업 완료.
19062 번 째 기업 완료.
19063 번 째 기업 완료.
19064 번 째 기업 완료.
19065 번 째 기업 완료.
19066 번 째 기업 완료.
19067 번 째 기업 완료.
19068 번 째 기업 완료.
19069 번 째 기업 완료.
19070

In [20]:
df_info.to_csv("company_info.csv", index = False)
print("저장완료")

저장완료
